In [12]:
import numpy as np 
from pathlib import Path
from aggrigator.optimized_gearys import fast_gearys_C
from aggrigator.methods import AggregationMethods as am
from aggrigator.optimized_gearys import fast_gearys_C
from aggrigator.uncertainty_maps import UncertaintyMap 


In [4]:
task = 'instance'
model_noise = 0
variation  = 'nuclei_intensity'
data_noise = '0_75'
decomp = 'pu'
uq_method = 'tta'

path = '/home/vanessa/Documents/data/uncertainty_arctique_v1-0-corrected_14/'
uq_path = Path(path).joinpath("UQ_maps")
metadata_path = Path(path).joinpath("UQ_metadata")

In [13]:
def preprocess_slice_spatial_measure(maps):
    """Ensures that hetmaps do not have negative values"""
    return np.clip(np.array(maps), 0, None)

In [14]:
# Load uncertainty maps
map_type = f"{task}_noise_{model_noise}_{variation}_{data_noise}_{uq_method}_{decomp}.npy"
map_file = uq_path.joinpath(map_type)
print(f"Loading uncertainty map: {map_type}")

uq_maps = preprocess_slice_spatial_measure(np.load(map_file))
    
# Load metadata indices
metadata_type = f"{task}_noise_{model_noise}_{variation}_{data_noise}_{uq_method}_{decomp}_sample_idx.npy"
metadata_file_path = metadata_path.joinpath(metadata_type)
indices = np.load(metadata_file_path)

Loading uncertainty map: instance_noise_0_nuclei_intensity_0_75_tta_pu.npy


In [15]:
uq_maps = [UncertaintyMap(array=array, mask=None, name=None) for array in uq_maps]
idx = 1

In [16]:
am.gearys_C(uq_maps[idx], None)

0.07213056640703336

In [18]:
fast_gearys_C(uq_maps[idx], None)

0.07219286125756112

In [31]:
0.07219286125756112 - 0.07213056640703336 < 1e-4

True

In [20]:
def test_gearys_implementation():
    """
    Test function to verify the fast implementation matches libpysal.
    """
    from esda.geary import Geary
    import libpysal
    
    # Create a small test array
    test_array = np.array([[1, 2, 3], 
                          [4, 5, 6],
                          [7, 8, 9]])
    test_map = UncertaintyMap(array=test_array, mask=None, name=None)
    
    # Compute using original libpysal method
    h, w = test_array.shape
    w_libpysal = libpysal.weights.lat2W(h, w)
    geary_original = Geary(test_array.ravel(), w_libpysal)
    
    # Compute using our fast method
    geary_fast = fast_gearys_C(test_map, None)
    
    print(f"Original Geary's C: {geary_original.C}")
    print(f"Fast Geary's C: {geary_fast}")
    print(f"Absolute difference: {abs(geary_original.C - geary_fast)}")
    
    return abs(geary_original.C - geary_fast) < 1e-10

In [21]:
test_gearys_implementation()

Original Geary's C: 0.3333333333333333
Fast Geary's C: 0.3333333333333333
Absolute difference: 0.0


True